In [2]:
import pickle as pkl
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
from itertools import chain

with open('Data/training_data.pkl', 'rb') as file:
    train_graphs = pkl.load(file)

In [3]:
#Alphabet instances for each graph over all hops
#List of N empty lists
def vertexHistogram(graphList):
    N = len(graphList)
    vertexlabs = np.zeros((N, 50+4))
    # edgelabs = np.zeros((N, 4))
    for i in tqdm(range(N)):
        va = nx.get_node_attributes(graphList[i], 'labels').values()
        va = list(chain.from_iterable(va))
        vb = np.c_[np.unique(va, return_counts=1)]
        vertexlabs[i, vb[:,0]] = vb[:,1]
        ea = nx.get_edge_attributes(graphList[i], 'labels').values()
        ea = list(chain.from_iterable(ea))
        eb = np.c_[np.unique(ea, return_counts=1)]
        if len(eb!=0): #completely disconnected graph
            vertexlabs[i,  50+eb[:,0]] = eb[:, 1]

    L = pd.DataFrame(vertexlabs, columns=[f'v{i}' for i in range(50)]+[f'e{i}' for i in range(4)])
    # print(L.head())
    return L#.dot(L.T)

VHKernel = vertexHistogram(train_graphs)
# np.savetxt('VHKernel.txt', VHKernel)

100%|██████████| 6000/6000 [00:01<00:00, 3930.34it/s]


Fit

In [4]:
VHKernel.shape

(6000, 54)

In [5]:
with open('Data/training_labels.pkl', 'rb') as file:
    labels = pkl.load(file)
VHData = VHKernel#np.loadtxt('VHKernel.txt')

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(VHData, labels, test_size=0.3, random_state=1)
X_train.shape

(4200, 54)

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lr = LogisticRegression(random_state=0, solver='newton-cholesky')
lr.fit(X_train, y_train)
pred = lr.predict(X_validate)
fpr, tpr, thresholds = metrics.roc_curve(y_validate, pred, pos_label=1)
print(f'AUC: {metrics.auc(fpr, tpr)}')

NameError: name 'X_train' is not defined

AUC: 0.6201100244498777
